**문자열 간 유사도**

1. jarowinkler 유사도 
- 두 문자열의 공통 문자와 이동거리를 기반으로 계산
- 문자열의 접두부가 일치하는 정도에 따라 가중치 부여 
- 값 : 0 ~ 1 (1에 가까울수록 동일함)

2. levenshtein 거리
- 두 문자열 간의 편집 거리 (삽입, 교체, 삭제 에 따라 가중치 다르게 부여 가능)
- 값이 작을수록 유사함

In [1]:
import numpy as np
from jarowinkler import jarowinkler_similarity
from difflib import SequenceMatcher

# 1. 각 메뉴끼리 jaro winkler 거리 계산해서 유사도 행렬 만들기 (prop1 : 순행 비율, prop2 : 역행 비율)
def calculate_similarity_matrix(strings, prop1, prop2): 

    n = len(strings)
    similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            similarity = jarowinkler_similarity(strings[i], strings[j]) * prop1 + jarowinkler_similarity(strings[i][::-1], strings[j][::-1]) * prop2
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity
    
    return similarity_matrix

# 2. 유사도가 임계값 이상인 것들만 그룹화하기 
def group_similar_words(similarity_matrix, words, threshold):

    n = len(words)
    groups = []
    used = set() # 이미 그룹처리한 메뉴의 index

    def possible_to_group(group, word_idx):
        return all(similarity_matrix[word_idx][g] >= threshold for g in group)
    
    for i in range(n):
        if i in used:
            continue
        
        group = [i]
        for j in range(i+1, n):
            if j in used:
                continue
            
            if possible_to_group(group, j):
                group.append(j)
                used.add(j)
        

        groups.append([words[idx] for idx in group])
    
    regular_groups = [group for group in groups if len(group) > 1]
    remainder = [group[0] for group in groups if len(group) == 1]

    return regular_groups, remainder

# 3. 그룹들에 대해서 키워드 추출해서 {키워드 : 그룹} 형태로 반환
def group_keyword(groups):

    def find_common_substrings(strings, threshold=0.5):
        if not strings:
            return ""
        
        # 가장 짧은 문자열을 기준
        base = min(strings, key=len)
        common = base
        
        for s in strings:
            new_common = ""
            for i in range(len(common)):
                for j in range(i+1, len(common)+1):
                    substring = common[i:j]
                    if len(substring) < 2:  # 너무 짧은 부분 문자열은 무시
                        continue
                    
                    # 현재 부분 문자열과 다른 문자열 사이의 유사도를 계산
                    matcher = SequenceMatcher(None, substring, s)
                    similarity = matcher.ratio()
                    
                    if similarity >= threshold and len(substring) > len(new_common):
                        new_common = substring
            
            common = new_common
            
            if not common:  # 공통 부분이 없으면 중단
                break
        
        return common
    
    keyword_dict = {}
    for menus in groups:
        keyword = find_common_substrings(menus) 
        keyword_dict[keyword] = menus 
    return keyword_dict

# 4. 키워드와 remainder 리스트에 있는 값들의 유사도 확인해서 가장 큰 유사도 가지는 그룹으로 처리

def remainder_grouping(remainder, keyword_dict : dict, threshold) :
    last_remainder = []
    for item in remainder:
        best_similarity = 0  # 초기값을 0으로 설정
        best_keyword = None
        for keyword in keyword_dict.keys():
            similarity = jarowinkler_similarity(keyword, item)
            if similarity > best_similarity:
                best_similarity = similarity
                best_keyword = keyword
        
        if best_similarity > threshold:
            keyword_dict[best_keyword].append(item)
        else:
            last_remainder.append(item)
    
    return keyword_dict, last_remainder

In [2]:
# 메뉴 데이터 모으기 
import pandas as pd
df1 = pd.read_csv('../../0_data/total.csv')['Menu']
df2 = pd.read_csv('../../0_data/total_menu.csv')['Menu']
df3 = pd.read_csv('../../0_data/24.08.04_01.13_울산.csv')['Menu']
df = pd.concat([df1, df2, df3])

data = []
for i in df:
    menus = i.split(',')
    for menu in menus:
        menu = menu.strip()
        if '&' in menu :
            menu = menu[:menu.index('&')]
        if '*' in menu :
            menu = menu[:menu.index('*')]
        if not menu:
            continue
    data.append(menu)


data = sorted(list(set(data)))
print(len(data)) # 12682 -> 1959 -> 1922

1922


In [3]:
# 첫번째 그룹화 

similarity_matrix = calculate_similarity_matrix(data, 0.4, 0.6)
regular_groups, remainder1 = group_similar_words(similarity_matrix, data, 0.8)
keyword_group1 = group_keyword(regular_groups)
total_group1, last_remainder1 = remainder_grouping(remainder1, keyword_group1, 0.7)

print(f'처음 실행 그룹 수 : {len(keyword_group1)}, 이후 남은 메뉴 개수 : {len(remainder1)}')
print(f'첫번째 그룹화 완료 : {len(total_group1)}')
print(f'첫번째 그룹화 완료 후 남은 메뉴 개수 : {len(last_remainder1)}')


처음 실행 그룹 수 : 400, 이후 남은 메뉴 개수 : 940
첫번째 그룹화 완료 : 400
첫번째 그룹화 완료 후 남은 메뉴 개수 : 531


In [4]:
for key, value in total_group1.items():
    print(f'keyword : {key} (총 {len(value)}), values : {value}')

keyword : 식목일케이크 (총 6), values : ["'식목일'초코케이크", '식목일케이크', '생일축하롤케이크', '식', '식목일케익', '식목일쿠키']
keyword : ㅐ추김치 (총 5), values : ['ㅐ추김치', '배추김치', '베추김치', '부추김치', '전배추김치']
keyword : 초콜릿 (총 3), values : ['가나초콜릿밀크', '초콜릿', '초콜렛']
keyword : 가래떡구이 (총 3), values : ['가래떡구이', '치즈가래떡구이', '가래떡꼬치']
keyword : 갈릭크로와상 (총 3), values : ['갈릭크로와상', '인절미크로와상', '크림크로와상']
keyword : 갈릭크림치즈피자 (총 2), values : ['갈릭크림치즈피자', '수제갈릭크림치즈피자']
keyword : 감귤음료 (총 5), values : ['감귤과즙음료', '감귤음료', '감', '감귤', '귤']
keyword : 오렌지주스 (총 10), values : ['감귤오렌지주스', '델몬트오렌지주스', '따옴오렌지주스', '레드오렌지주스', '오렌지망고주스', '오렌지자몽주스', '오렌지주스', '오렌지주스^', '착즙오렌지주스', '파인오렌지주스']
keyword : 감귤주스 (총 4), values : ['감귤주스', '감귤주스^', '감귤쥬스', '배주스']
keyword : 감귤푸딩 (총 2), values : ['감귤푸딩', '밀감푸딩']
keyword : 한라봉 (총 5), values : ['감귤한라봉주스', '감귤한라봉쥬스', '한라봉', '한라', '한라봉미니롤케익']
keyword : 감자핫도그 (총 8), values : ['감자모짜찰핫도그', '감자핫도그', '꼬마핫도그', '뉴욕핫도그', '뿌링핫도그', '연잎핫도그', '크런치핫도그', '핫도그/케찹']
keyword : 감자에그샌드위치 (총 3), values : ['감자에그샌드위치', '모닝빵감자샌드위치', '에그드랍샌드위치']
keyword 

In [5]:
# 두번째 그룹화 
similarity_matrix = calculate_similarity_matrix(last_remainder1, 0.4, 0.6)
regular_groups, remainder2 = group_similar_words(similarity_matrix, last_remainder1, 0.75)
keyword_group2 = group_keyword(regular_groups)
total_group2, last_remainder2 = remainder_grouping(remainder2, keyword_group2, 0.65)

print(f'두번째 실행 그룹 수 : {len(keyword_group2)}, 이후 남은 메뉴 개수 : {len(remainder2)}')
print(f'두번째 그룹화 완료 : {len(total_group2)}')
print(f'두번째 그룹화 완료 후 남은 메뉴 개수 : {len(last_remainder2)}')

두번째 실행 그룹 수 : 25, 이후 남은 메뉴 개수 : 478
두번째 그룹화 완료 : 25
두번째 그룹화 완료 후 남은 메뉴 개수 : 459


In [6]:
for key, value in total_group2.items():
    print(f'keyword : {key} (총 {len(value)}), values : {value}')

keyword : 약과타르트 (총 5), values : ['가나슈타르트', '약과타르트', '호두타르트', '꿀약과', '약과']
keyword : 계란토스트 (총 3), values : ['계란토스트', '전남친토스트', '프렌치토스트']
keyword : 깨송편 (총 2), values : ['깨송편', '식혜송편']
keyword : 냉식혜 (총 4), values : ['냉식혜', '비락식혜', '수제식혜', '전통식혜']
keyword : 대왕꽈배기 (총 2), values : ['대왕꽈배기', '찹쌀꽈배기']
keyword : 도너츠 (총 3), values : ['도너츠', '도넛츠', '믹스너츠']
keyword : 스승의날케이크 (총 3), values : ['독도의 날 초코케이크', '스승의날케이크', '호국의달 케이크']
keyword : 딸기퐁당호두 (총 2), values : ['딸기퐁당아몬드', '딸기퐁당호두']
keyword : 레몬슬러쉬 (총 3), values : ['레몬슬러쉬', '망고슬러쉬', '녹여먹는슬러쉬']
keyword : 솜사탕 (총 2), values : ['막대사탕', '솜사탕']
keyword : 컵케익 (총 2), values : ['머핀케익', '컵케익']
keyword : 메로나음료 (총 3), values : ['메로나음료', '메로나튜브', '메로나']
keyword : 메론 (총 3), values : ['메론', '수제메론소다', '멜론']
keyword : 모듬부 (총 3), values : ['모듬부', '모듬쌈', '모듬양념류']
keyword : 케익 (총 2), values : ['미니케익..', '케익']
keyword : 반달단무지 (총 3), values : ['반달단무지', '슬림단무지', '하트단무지']
keyword : 쁘띠첼 (총 2), values : ['쁘띠첼', '쁘띠쿨']
keyword : 호떡 (총 4), values : ['설빙호떡..', '호떡', '꿀떡', '호떡

In [7]:
# total_group1과 total_group2가 같은 그룹의 키워드를 가지는지 확인
if len(total_group1) + len(total_group2) == len(set(list(total_group1.keys())+list(total_group2.keys()))):
    print('중복된 키워드 없습니다')

total_group1.update(total_group2)
menu_group = total_group1
print(len(menu_group))

중복된 키워드 없습니다
425


In [8]:
for key, value in menu_group.items():
    print(f'키워드 : {key}, 메뉴 : {value}')

키워드 : 식목일케이크, 메뉴 : ["'식목일'초코케이크", '식목일케이크', '생일축하롤케이크', '식', '식목일케익', '식목일쿠키']
키워드 : ㅐ추김치, 메뉴 : ['ㅐ추김치', '배추김치', '베추김치', '부추김치', '전배추김치']
키워드 : 초콜릿, 메뉴 : ['가나초콜릿밀크', '초콜릿', '초콜렛']
키워드 : 가래떡구이, 메뉴 : ['가래떡구이', '치즈가래떡구이', '가래떡꼬치']
키워드 : 갈릭크로와상, 메뉴 : ['갈릭크로와상', '인절미크로와상', '크림크로와상']
키워드 : 갈릭크림치즈피자, 메뉴 : ['갈릭크림치즈피자', '수제갈릭크림치즈피자']
키워드 : 감귤음료, 메뉴 : ['감귤과즙음료', '감귤음료', '감', '감귤', '귤']
키워드 : 오렌지주스, 메뉴 : ['감귤오렌지주스', '델몬트오렌지주스', '따옴오렌지주스', '레드오렌지주스', '오렌지망고주스', '오렌지자몽주스', '오렌지주스', '오렌지주스^', '착즙오렌지주스', '파인오렌지주스']
키워드 : 감귤주스, 메뉴 : ['감귤주스', '감귤주스^', '감귤쥬스', '배주스']
키워드 : 감귤푸딩, 메뉴 : ['감귤푸딩', '밀감푸딩']
키워드 : 한라봉, 메뉴 : ['감귤한라봉주스', '감귤한라봉쥬스', '한라봉', '한라', '한라봉미니롤케익']
키워드 : 감자핫도그, 메뉴 : ['감자모짜찰핫도그', '감자핫도그', '꼬마핫도그', '뉴욕핫도그', '뿌링핫도그', '연잎핫도그', '크런치핫도그', '핫도그/케찹']
키워드 : 감자에그샌드위치, 메뉴 : ['감자에그샌드위치', '모닝빵감자샌드위치', '에그드랍샌드위치']
키워드 : 감자튀김, 메뉴 : ['감자튀김', '웨지감자튀김']
키워드 : 갓김치, 메뉴 : ['갓김치', '물김치', '백김치', '파김치', '김치찌개', '깻잎김치', '동백김치', '보쌈김치', '볶음김치']
키워드 : 갓김치배추김치, 메뉴 : ['갓김치배추김치', '명란김구이/배추김치']
키워드 : 강화요구르트, 메뉴 : ['강화요

실제 csv 파일에 적용

In [24]:
import csv

def convert_menu_to_group(df, output_file, menu_group):
    with open(output_file,'w',newline='', encoding='utf-8')as outfile:
        writer = csv.writer(outfile)

        for i in df['Menu']:
            menu_row = i.split(',')            
            new_row = []
            for item in menu_row:
                # 메뉴 항목을 그룹으로 변환 (있는 경우)
                found = False
                for group, menus in menu_group.items():
                    if item in menus:
                        new_row.append(group)
                        found = True
                        break
                # 변환되지 않은 항목은 그대로 유지
                if not found:                
                    new_row.append(item)
            writer.writerow(new_row)


In [25]:
df1 = pd.read_csv('../../0_data/total_menu.csv')
convert_menu_to_group(df1, 'df1_new.csv',menu_group)